# 다국어 질의 대응 네이버 검색 요약 에이전트 개발

> 목표 : 사용자 정의 도구 개발 및 Agent 구현

## 환경 설정

In [1]:
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

True

In [2]:
import os
import json

from pprint import pprint

In [3]:
from langfuse.langchain import CallbackHandler

# 콜백 핸들러 생성
langfuse_handler = CallbackHandler()

## 1. 도구 정의

### 1\) 언어 감지 및 번역 자동화 도구

In [48]:
import deepl
from langdetect import detect

# Deepl 번역기 생성
translator = deepl.Translator(os.getenv('DEEPL_API_KEY'))

def detect_and_translate(query, target_lang='KO'):
    """ 텍스트의 언어 감지 및 목표 언어로 번역 수행 """
    # 언어 감지
    detected_lang = detect(query)
    source_lang = "zh" if "zh" in detected_lang else detected_lang

    # 언어가 목표 언어와 다른 경우 번역
    if source_lang.upper() != target_lang:
        translated_query = translator.translate_text(
            query,
            source_lang=source_lang.upper(),
            target_lang=target_lang.upper()
        )
    
    return str(translated_query), source_lang

In [5]:
# 테스트
query = "I heard they're building a data center these days. Could you summarize the related news?"
translated_query, source_lang = detect_and_translate(query, target_lang='KO')

print(f"Source language: {source_lang}")
print(f"Translated query: {translated_query}")

Source language: en
Translated query: 요즘 데이터 센터를 짓고 있다고 들었습니다. 관련 소식을 요약해 주시겠어요?


### 2\) 네이버 검색 도구

In [6]:
import requests, os
from langchain_core.tools import tool
from typing import Dict

def naver_search(api: str, keyword: str, display: int = 10) -> Dict:
    """ 네이버 검색 API 공통 요청 함수 """
    base_url = "https://openapi.naver.com/v1/search/"
    url = f"{base_url}{api}.json"

    headers = {
        "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
        "X-Naver-Client-Secret": os.getenv("NAVER_CLIENT_SECRET")
    }
    params = {
        "query": keyword,
        "display": display
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"[{api}] status_code:", response.status_code)
        return {}

# 도구화: 뉴스
@tool
def naver_news_search(keyword: str) -> Dict:
    """ 네이버 검색 API를 사용하여 뉴스 검색 결과를 조회 """
    return naver_search("news", keyword)

# 도구화: 블로그
@tool
def naver_blog_search(keyword: str) -> Dict:
    """ 네이버 검색 API를 사용하여 블로그 검색 결과를 조회 """
    return naver_search("blog", keyword)

# 도구화: 전문자료
@tool
def naver_doc_search(keyword: str) -> Dict:
    """ 네이버 검색 API를 사용하여 전문자료 검색 결과를 조회 """
    return naver_search("doc", keyword)

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 체인 구성
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 검색 전문가입니다. 사용자의 질의를 분석해 적절한 검색 도구를 선택하세요."),
    ("human", "사용자 질의: {query}")
])
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.1
)
# 도구 목록 생성 
tools = [naver_news_search, naver_blog_search, naver_doc_search]

# 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools)

# 에이전트 실행기 생성
tool_select_chain = prompt | llm_with_tools

In [8]:
# 테스트
response = tool_select_chain.invoke({"query": translated_query})
response.tool_calls

[{'name': 'naver_news_search',
  'args': {'keyword': '데이터 센터 건설'},
  'id': 'call_nUYKnlpkeuRtbaJPHAzjU4Ll',
  'type': 'tool_call'}]

### 3\) 웹 본문 크롤링 도구

In [10]:
from langchain_community.document_loaders import WebBaseLoader

@tool
def web_loader(url: str) -> str:
    """ 웹 본문 크롤링 함수 """
    loader = WebBaseLoader(url)
    docs = loader.load()
    return " ".join(docs[0].page_content.split())

USER_AGENT environment variable not set, consider setting it to identify your requests.


## 2. 체인 정의

### 1\) 키워드 추출 체인

In [49]:
from pydantic import BaseModel, Field
from typing import List, Dict
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

class KeywordList(BaseModel):
    keywords: List[str] = Field(..., description="뉴스 검색을 위한 핵심 키워드 리스트")
    
system_message = """당신은 검색 전문가입니다.
사용자가 입력한 질의에서 네이버 검색에 사용할 수 있는 핵심 키워드를 추출합니다.

- 키워드는 5개 이내로 추출합니다.
- 중복되거나 의미가 약한 단어는 제외합니다.
- 검색 키워드로 **부적절한 단어('정보', '소식', '기사', '이슈', '내용', '현황' 등)은 제외**합니다.

예)
질문: "요즘 중국이 반도체 산업에서 미국과 어떤 갈등을 겪고 있나요? 최근 뉴스 좀 요약해주세요."
키워드: ['중국', '미국', '반도체']"""

# 체인 구성
prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", "질문: {query}\n키워드: ")
])
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.1
)
keyword_chain = prompt | llm.with_structured_output(KeywordList)

@chain
def run_extract_keywords(query:str, config: RunnableConfig) -> Dict:
    """ 키워드 추출 체인 정의 """
    # 입력 쿼리 언어 감지
    original_lang = detect(query)

    # 한국어가 아닌 경우 번역
    if original_lang.upper() != 'KO':
        translated_query, source_lang = detect_and_translate(query, target_lang='KO')
    else:
        translated_query = query
        source_lang = original_lang

    return {
        "query": translated_query,
        "source_lang": source_lang,
        "keywords": keyword_chain.invoke(
            {"query": translated_query},
            config=config
        ).keywords
    }

In [41]:
# 테스트
query = "I heard they're building a data center these days. Could you summarize the related news?"
keywords = run_extract_keywords.invoke(query, config={"callbacks":[langfuse_handler]})
keywords

{'query': '요즘 데이터 센터를 짓고 있다고 들었습니다. 관련 소식을 요약해 주시겠어요?',
 'source_lang': 'en',
 'keywords': ['데이터 센터', '건설']}

### 2\) 뉴스 추출 및 요약 체인

In [28]:
from sklearn.metrics.pairwise import cosine_similarity
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# LLM 정의
llm = ChatOpenAI(model="gpt-4.1", temperature=0)

# 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 검색 결과를 간결하게 요약하는 검색 요약 전문가입니다."),
    ("human", "다음 검색 내용을 간단하고 핵심적으로 {source_lang} 언어로 요약해 주세요:\n\n{article}")
])

# 체인 정의
summary_chain = prompt | llm

@chain
def run_summarize(user_input, config: RunnableConfig):
    """ 키워드 기반 검색 및 의미 기반 유사도 정렬 """
    query = user_input["query"]
    source_lang = user_input["source_lang"]
    keywords = user_input["keywords"]
    embedding_model = user_input["embedding_model"]

    tool_calls = tool_select_chain.invoke(
        {"query": query},
        config=config
    )

    article_previews = []
    article_links = []
    
    for call in tool_calls.tool_calls:
        tool_name = call["name"]

        selected_tool = {
            "naver_news_search": naver_news_search,
            "naver_blog_search": naver_blog_search,
            "naver_doc_search": naver_doc_search
        }.get(tool_name)

        if not selected_tool:
            continue

        # 키워드별 자료 수집
        for keyword in keywords:
            news = selected_tool.invoke(
                {"keyword": keyword},
                config=config
            )
            for item in news["items"]:                
                combined_text = item["title"] + " " + item["description"]
                article_previews.append(combined_text)
                article_links.append(item["link"])

    # 질의와 뉴스 텍스트 임베딩
    query_embedding = embedding_model.embed_query(query)
    article_embeddings = embedding_model.embed_documents(article_previews)

    # 유사도 계산
    sims = cosine_similarity([query_embedding], article_embeddings)[0]

    # 유사도 기준 정렬
    ranked = sorted(zip(sims, article_links, article_previews), key=lambda x: x[0], reverse=True)

    # 상위 5개 결과 반환
    top5 = ranked[:5]

    # 본문 내용 크롤링
    summaries = []
    for score, link, text in top5:
        article = web_loader.invoke(link)
        summary = summary_chain.invoke(
            {"article": article, "source_lang": source_lang},
            config=config
        )
        summaries.append({
            "summary": summary.content,
            "text": text,
            "score": round(float(score), 4),
            "link": link
        })
    
    return {
        "query": query,
        "articles": summaries
    } 

In [16]:
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings 
from langchain_ollama import OllamaEmbeddings

from huggingface_hub import login

load_dotenv()

HUGGINGFACE = os.environ["HUGGINGFACE_TOKEN_KEY"]
login(HUGGINGFACE)

# 임베딩 정의
embeddings_openai = OpenAIEmbeddings(model="text-embedding-3-small")
embeddings_huggingface = HuggingFaceEmbeddings(
    model_name="Alibaba-NLP/gte-multilingual-base",
    model_kwargs={"trust_remote_code": True}
)
embeddings_ollama = OllamaEmbeddings(model="nomic-embed-text")

/Users/sunhwaryu/Documents/llm-study/prj03/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Overriding of current TracerProvider is not allowed
Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
# 임베딩 퀄리티 확인
inputs = keywords.copy()

inputs["embedding_model"] = embeddings_openai
result_openai = run_summarize.invoke(inputs, config={"callbacks": [langfuse_handler]})
print("OpenAI: ")
for news in result_openai["articles"]:
    pprint(news)

inputs["embedding_model"] = embeddings_huggingface
result_huggingface = run_summarize.invoke(inputs, config={"callbacks": [langfuse_handler]})
print("\n\nAlibaba: ")
for news in result_huggingface["articles"]:
    pprint(news)

inputs["embedding_model"] = embeddings_ollama
result_ollama = run_summarize.invoke(inputs, config={"callbacks": [langfuse_handler]})
print("\n\nOllama: ")
for news in result_ollama["articles"]:
    pprint(news)

OpenAI: 
{'link': 'https://n.news.naver.com/mnews/article/011/0004505679?sid=105',
 'score': 0.4192,
 'summary': 'The Korean government will invest 27.3 billion KRW to build '
            'small-scale micro data centers (MDCs) in regional areas such as '
            'Daegu and Gwangju, aiming to expand the domestic AI semiconductor '
            'ecosystem. These MDCs, equipped with Korean AI chips, will '
            'provide affordable AI computing services to SMEs and medical '
            'institutions, and offer domestic chipmakers real-world '
            'application opportunities. The project seeks to reduce reliance '
            'on global GPU providers like Nvidia and address the concentration '
            'of data centers in the Seoul metropolitan area. The initiative '
            'will run from 2025 to 2029.',
 'text': "정부, 지방 소형 <b>데이터센터</b> 구축…'국산 AI 반도체' 판 키운다 정부가 "
         '마이크로<b>데이터센터</b>(MDC)를 구축해 국산 인공지능(AI) 반도체 생태계 확대에 나선다. 국산 AI 반도체 기반 '
         '소규모 <b>데이터센터

## 3. 최종 체인 구성 및 테스트

In [50]:
from langchain_core.runnables import RunnableLambda

map_inputs = RunnableLambda(lambda x: {
    "query": x["query"],
    "source_lang": x["source_lang"],
    "keywords": x["keywords"],
    "embedding_model": embeddings_huggingface
})

final_chain = run_extract_keywords | map_inputs | run_summarize

In [ ]:
# 중국어 번역이 잘 안됨
test_set = [
    "요즘 데이터 센터 관련 기술 동향이 어떻게 되나요?",
    "What are the recent trends in electric vehicles in Korea?",
    "最近韩国在人工智能方面有什么新进展？",
    "韓国のスタートアップ業界の現状について教えてください。",
    "韓國目前在綠色能源領域有哪些重要政策？"
]

In [38]:
final_chain.invoke(test_set[0], config={"callbacks": [langfuse_handler]})

{'query': '요즘 데이터 센터 관련 기술 동향이 어떻게 되나요?',
 'articles': [{'summary': "뉴타닉스는 VM웨어 인수 이후 높아진 라이선스 비용 등으로 '탈 VM웨어' 수요가 늘어난 시장에서, 간편하고 안정적인 전환 솔루션을 제공한다고 강조했습니다. 멀티·하이브리드 클라우드, AI 환경, 애플리케이션 현대화, 대형언어모델(LLM) 관리까지 하나의 플랫폼에서 지원하며, 예측 가능한 라이선스 비용과 높은 사용 편의성, 엔터프라이즈급 안정성을 강점으로 내세웠습니다. 또한, 오픈소스 대비 운영 관리의 복잡성을 줄이고, 다양한 파트너십과 AI 지원을 강화해 한국 시장에서도 적극적으로 투자와 협력을 확대할 계획입니다.",
   'text': '뉴타닉스 “간편하고 안정적인 탈VM웨어·AX 해답 제시” 하이퍼컨퍼지드인프라(HCI) 분야를 대표하는 이 글로벌 기업은 최근 들어 멀티·하이브리드 클라우드 지원을 강화하면서 소프트웨어정의<b>데이터센터</b>(SDDC)를 위한 솔루션을 고도화하고 있다. 2023년 브로드컴이 VM웨어를... ',
   'score': 0.7003,
   'link': 'https://n.news.naver.com/mnews/article/029/0002966377?sid=105'},
  {'summary': '정부가 273억 원을 투입해 대구, 광주 등 비수도권에 국산 AI 반도체 기반 소형 데이터센터(MDC)를 구축한다. 이를 통해 중소기업·의료기관 등에 저렴한 AI 연산 서비스를 제공하고, 국산 AI 반도체 기업의 실증 및 수요처 확보를 지원할 계획이다. 기존 글로벌 기업 중심의 데이터센터 시장 구조를 개선하고, 국산 AI 반도체 생태계 확대를 목표로 한다.',
   'text': "정부, 지방 소형 <b>데이터센터</b> 구축…'국산 AI 반도체' 판 키운다 정부가 마이크로<b>데이터센터</b>(MDC)를 구축해 국산 인공지능(AI) 반도체 생태계 확대에 나선다. 국산 AI 반도체 기반 소규모 <b>데이터센터</b

In [42]:
final_chain.invoke(test_set[1], config={"callbacks": [langfuse_handler]})

{'query': '최근 국내 전기자동차의 트렌드는 무엇인가요?',
 'articles': [{'summary': 'Naver blog of the Korea Agency for Infrastructure Technology Advancement (KAIA), sharing news, research, and updates on national land, transport, and infrastructure technology.',
   'text': '<b>전기자동차</b>가 위험하다고? 이제 안전할거야! <b>전기자동차</b>는 배터리로부터 전기에너지를 전기모터로 공급하여 구동력을 발생시키는 자동차로, 무공해 친환경 교통수단입니다. ‘EV’라고 불리기도 해요. 전기에너지를 활용하다 보니 이산화탄소를... ',
   'score': 0.6569,
   'link': 'https://blog.naver.com/kaia_pr/223699953596'},
  {'summary': 'Summary:  \nA Naver blog post shares stories and information about the freshwater fish "bitterling" (버들붕어), including its characteristics, habitat, and personal experiences related to catching or observing the fish.',
   'text': '쉐보레 블레이저 EV, <b>국내</b> 출시되나? 북미에서는 이미 출시되어 판매 중이지만, <b>국내</b>에서는 아직 만나볼 수 없는 차량입니다. 한국지엠은 블레이저 EV의 <b>국내</b> 도입 가능성을 언급한 바 있으며, 2025년까지 얼티엄(Ultium) 플랫폼 기반 전기차... ',
   'score': 0.5954,
   'link': 'https://blog.naver.com/djusti/223916623792'},
  {'summary': "Mer's Blog: A 

In [ ]:
final_chain.invoke(test_set[2], config={"callbacks": [langfuse_handler]}) # 중국어 번역이 잘 안됨

{'query': '最近韩国在人工智能方面有什么新进展？',
 'articles': [{'summary': "네이버 블로그 'Carpediem :)'는 일상, 여행, 취미 등 다양한 주제의 글을 공유하는 개인 블로그입니다.",
   'text': '<b>韩国</b> <b>韩国</b> (한궈) - 한국 <b>韩国</b>人 (한궈런) - 한국인 <b>韩国</b>语 (한궈위) - 한국어 大韩民国 (다한밍궈) - 대한민국 中国人 (중궈런) 關係 (꽌시) 관계 韓中關係 (한중꽌시) 한중관계 ※ 비공식적 명칭: 寒国 (한궈) 寒国(한궈)가... ',
   'score': 0.6276,
   'link': 'https://blog.naver.com/antitrust7/223608572807'},
  {'summary': "이 논문은 1985년부터 2018년까지 국내 인공지능(AI) 분야 학술논문 1,691편을 대상으로 토픽모델링과 의미연결망분석을 활용해 연구동향을 분석한 것이다. 주요 연구 주제로 '인공지능과 인간', '인공지능과 기술'이 도출되었으며, 현재는 기술 자체에 초점을 둔 연구가 많으나, 앞으로는 인공지능과 인간, 인문학 등 다양한 융합 분야에 대한 심층 연구가 필요함을 시사한다.",
   'text': "국내 <b>인공지능</b>분야 연구동향 분석 : 토픽모델링과 의미연결망분석을 중 An analysis of artificial intelligence(A.I.)_related studies' trends in Korea focused on topic modeling and semantic network analysis ",
   'score': 0.6171,
   'link': 'https://academic.naver.com/article.naver?doc_id=606314593'},
  {'summary': '린센세가 운영하는 네이버 블로그로, 일본어와 중국어 학습을 재미있게 소개하고 다양한 언어 팁과 정보를 제공함.',
   'text': '한국은 한궈（<b

In [52]:
final_chain.invoke(test_set[3], config={"callbacks": [langfuse_handler]})

{'query': '한국의 스타트업 업계 현황에 대해 말씀해 주세요.',
 'articles': [{'summary': 'NAVERの学術情報サービスでは、さまざまな学術論文やジャーナル（例：IEEE、Elsevier、Springerなど）の検索・閲覧が可能です。論文は年別・巻号別に整理されており、ISSNやDOIなどで管理されています。韓国語・英語対応、DBpiaやKISTIなど外部データベースとも連携しています。',
   'text': '한국<b>스타트업</b>생태계의 현황과 과제   목차\n한국<b>스타트업</b>생태계의 현황과 과제 / 임정욱 1\n [머리글] 1\n 벤처캐피탈의 새로운 트랜드-마이크',
   'score': 0.7282,
   'link': 'https://academic.naver.com/article.naver?doc_id=176512679'},
  {'summary': '本論文は、韓国における起業初期企業の高い失敗率を低減するための戦略として「リーンスタートアップ」手法を紹介し、その基本フレームワークと主要原則、段階別の適用方法、導入時の留意点を事例と共に解説しています。リーンスタートアップは、起業家・教育者・メンターにとって有効なプロセスであり、起業成功率向上に寄与する包括的な視点を提供しています。',
   'text': "'린 <b>스타트업</b>' : 창업 초기기업의 실패 최소화 전략 ‘Lean Startup’ : The way to Reduce the Failure Rate of Startups 근 한국경제의 화두가 되고 있는 창업활성화를 위",
   'score': 0.6839,
   'link': 'https://academic.naver.com/article.naver?doc_id=181888270'},
  {'summary': '韓国スタートアップの現状と成功要因を、ユニコーン企業である「ウワハン兄弟（Woowa Brothers）」の事例を通じて分析した研究。ERISモデル（起業家、資源、産業環境、戦略）と成長段階別モデルを用い、ウワハン兄弟の成功要因として、創業者の特性、独自の企業文化、差別化戦略、プラ

In [ ]:
final_chain.invoke(test_set[4], config={"callbacks": [langfuse_handler]}) # 중국어 번역이 잘 안됨

{'query': '韓國目前在綠色能源領域有哪些重要政策？',
 'articles': [{'summary': '전 세계적으로 기후변화와 화석연료 부족 문제로 인해 각국이 녹색에너지(태양광, 풍력, 바이오연료 등) 개발에 주력하고 있습니다. 각국의 발전 중점 분야는 국가별 조건에 따라 다르며, 주요 과제는 국가 에너지 정책의 지원, 기술 개발, 비용 절감입니다. 특히 대만은 에너지 자립과 온실가스 감축을 위해 자국에 적합한 녹색에너지 산업과 핵심 기술 확보가 필요합니다.',
   'text': '各國<b>綠色能源</b>發展重點項目以及未來的挑戰 The Important Development Items of Green Energy and Challenge in Each Country 由於全球暖化所造成的氣候變遷以及石化燃',
   'score': 0.6819,
   'link': 'https://academic.naver.com/article.naver?doc_id=84570339'},
  {'summary': '‘정이세상. 심정즉정도’는 마음이 바르면 바른 길을 간다는 의미를 담고 있습니다.',
   'text': '호주는 담수 없이 &quot;녹색 수소&quot;를 생산할 수 있는 &quot;해수에서...  最具減碳趨勢的<b>綠色能源</b>。自19世紀迄今，氫氣已廣泛用於汽車、飛行船以及太空梭的燃料供應。 文：張瑞邦（Tucker Chang） 澳洲阿得雷德大學（University of Adelaide）國際能源研究小組近日公布研究成果，... ',
   'score': 0.6722,
   'link': 'https://blog.naver.com/finemind/223039210705'},
  {'summary': '2024년 미국 대선 후 트럼프가 재집권할 경우, 미국의 녹색에너지(재생에너지) 정책은 크게 영향을 받을 전망입니다. 트럼프는 바이든 정부의 일부 에너지 법안을 철회해 미국의 기후 목표 달성이 어려워질 수 있습니다. 다만, 양당의 지지를 받는 일부 핵심 에너지는 정부 보조가